# Jupyter Notebook AI Assistant

> Your AI-powered coding companion for Jupyter notebooks

Build on top of the [Pydantic AI framework](https://ai.pydantic.dev/), and inspired by amazing [nbdev](https://nbdev.fast.ai/) library and it's creators, this library features:
- AI powered chat with your custom agent inside the notebook.
- By defauld agents will: 
  - have a memory of the code you have written so far.
  - create new code based on your prompt
- Default agent is provided with a basic system prompt and tools.
- For more details on how to customize the assistant see the [Pydantic AI documentation](https://ai.pydantic.dev/).
- Add new tools to the assistant


## Installation

```bash
pip install git+https://github.com/ndendic/jupyter-assistant.git
```


## Configuration

1. Set `PYDANTIC_AI_MODEL` in your environment variables
2. Set your model API key - ie. `OPENAI_API_KEY`

### 2. Usage

Import the jupyter_assistant.core module:


In [ ]:
from jupyter_assistant.core import *

PydanticAI is using model: openai:gpt-4o


<module>:8: No type or annotation for returned value 1


Start prompting by adding the `%%prompt` cell magic:

In [ ]:
%%prompt
Hello, how are you?

Hello! I'm here and ready to help you with anything you need. How can I assist you today?

jupyter_assistant.core works with default notebook_agent that has basic create_cell tool and short system_prompt. 

In [ ]:
from IPython.display import display, Markdown

display(Markdown(system_prompt))
notebook_agent


You are a helpful assistant that operates in a Jupyter notebook.
Your regular text responses are rendered as cell output.
You can create new cells, edit existing cells, and run code.
You can also use tools to help you with your tasks.
Today's date is 2024-12-31.


Agent(model=OpenAIModel(model_name='gpt-4o'), name='agent', end_strategy='early', model_settings=None)

Let's see how you can create a custom agent and set it as current agent.
Here we'll use `model` we set earlier and `set_agent` function to set it as current agent.

In [ ]:
#| exclude
from pydantic_ai import Agent

# Configure with custom system prompt
custom_prompt = system_prompt + """
Please always add emojis to your responses where possible.
You use joining-teacher style of speech but also try to be concise.
"""
my_agent = Agent(model, system_prompt=custom_prompt)

# Set as current agent
set_agent(my_agent)

Agent(model=OpenAIModel(model_name='gpt-4o'), name=None, end_strategy='early', model_settings=None)

In [ ]:
%%prompt
Hello, what are the things you can do for me?

Hello! I can assist you with various tasks related to working within a Jupyter notebook. Here are some things I can do for you:

1. **Create and Manage Cells**: I can create new code or markdown cells, edit existing cells, and run code in the notebook.

2. **Code Assistance**: I can help you write, debug, or improve your code. I can also provide explanations and documentation for certain functions or libraries.

3. **Data Analysis**: I can assist with data analysis tasks, such as data cleaning, visualization, and applying statistical or machine learning models.

4. **Tool Integration**: I can help integrate different tools and libraries into your workflow, such as setting up machine learning models or connecting to APIs.

5. **Answer Questions and Provide Information**: I can provide information on a wide range of topics, answer questions, and offer insights based on the content we're working on.

6. **Environment Setup**: I can guide you in setting up your Python environment, packages, and configurations needed for your projects.

Feel free to ask if you have a specific request or need assistance with anything else!

`last_prompt_result` variable will be automatically created and will contain the result of the prompt but also some other useful information.

In [ ]:
last_prompt_result.all_messages()

[ModelRequest(parts=[UserPromptPart(content="Here is the context of new notebook cells that were added:\nCell[1]:\nSource:\n## Installation\n\n```bash\npip install git+https://github.com/ndendic/jupyter-assistant.git\n```\n\nOutputs:\nNone\n\nCell[2]:\nSource:\n## Configuration\n\n1. Set `PYDANTIC_AI_MODEL` in your environment variables\n2. Set your model API key - ie. `OPENAI_API_KEY`\nOutputs:\nNone\n\nCell[3]:\nSource:\n### 2. Usage\n\nImport the jupyter_assistant.core module:\n\nOutputs:\nNone\n\nCell[4]:\nSource:\nfrom jupyter_assistant.core import *\nOutputs:\n[{'name': 'stdout', 'output_type': 'stream', 'text': ['PydanticAI is using model: openai:gpt-4o\\n']}, {'name': 'stderr', 'output_type': 'stream', 'text': ['<module>:8: No type or annotation for returned value 1\\n']}]\n\nCell[5]:\nSource:\nStart prompting by adding the `%%prompt` cell magic:\nOutputs:\nNone", timestamp=datetime.datetime(2024, 12, 31, 13, 30, 24, 475626, tzinfo=datetime.timezone.utc), part_kind='user-prompt

From the example above, you can see that in addition to the standard system prompt, agent will also get the context that provides notebook cells created so far with the whole chat history.

This context provides all the cells since the notebook was opened and any cells that we created between the last prompt we run and the current prompt.

This is useful for creating more complex agents that can use the history of the conversation to create more accurate responses.

Now let's ask our agent to create us some code.

In [ ]:
%%prompt
can you create me a pydantic model to store User data?

I have created a new code cell with a Pydantic model to store User data. You can find it in the notebook and modify it as needed. If there's anything else you need, feel free to ask!

In [ ]:
from pydantic import BaseModel

class User(BaseModel):
    username: str
    email: str
    full_name: str
    age: int
    active: bool = True

# Example of creating a User instance
user = User(username='johndoe', email='johndoe@example.com', full_name='John Doe', age=30)
print(user)


username='johndoe' email='johndoe@example.com' full_name='John Doe' age=30 active=True


Let's check the last 3 messages to see how agent utilized the tool.

In [ ]:
last_prompt_result.all_messages()[-3:]

[ModelResponse(parts=[ToolCallPart(tool_name='create_cell', args=ArgsJson(args_json='{"content":"from pydantic import BaseModel\\n\\nclass User(BaseModel):\\n    username: str\\n    email: str\\n    full_name: str\\n    age: int\\n    active: bool = True\\n\\n# Example of creating a User instance\\nuser = User(username=\'johndoe\', email=\'johndoe@example.com\', full_name=\'John Doe\', age=30)\\nprint(user)\\n","cell_type":"code"}'), tool_call_id='call_r8zXssuPEK4boFcwd1maO6yl', part_kind='tool-call')], timestamp=datetime.datetime(2024, 12, 31, 13, 30, 30, tzinfo=datetime.timezone.utc), kind='response'),
 ModelRequest(parts=[ToolReturnPart(tool_name='create_cell', content="Created new code  with content: from pydantic import BaseModel\n\nclass User(BaseModel):\n    username: str\n    email: str\n    full_name: str\n    age: int\n    active: bool = True\n\n# Example of creating a User instance\nuser = User(username='johndoe', email='johndoe@example.com', full_name='John Doe', age=30)\np

Let's ask our agent to summarize our conversation so far.

In [ ]:
%%prompt
Can you please summarize our conversation so far?

Certainly! Here's a summary of our conversation so far:

1. **Notebook Environment Setup**:
   - We discussed installing and configuring the Jupyter Assistant to enhance the interactive coding experience in a Jupyter notebook.

2. **Agent Functionality**:
   - The context provided includes how to work with the `jupyter_assistant.core` including setting up agents and utilizing advanced features.
   - We set up a custom agent using a model and a system prompt.
   
3. **Cell Additions**:
   - A Pydantic model for storing user data was added, showcasing how to structure a `User` class with attributes like `id`, `name`, `email`, and `is_active`.
   - Example instantiation of a `User` object was provided.

4. **Agent Interactions**:
   - I demonstrated how the agent can manage cell creation in the notebook.
   - We created a Pydantic model code cell as an example of utilizing the tool.

5. **Interactive Assistance**:
   - I outlined the various capabilities I can provide, such as creating cells, debugging code, assisting with data analysis, and providing explanations.

If you have any more questions or need further assistance, feel free to ask! 😊

If there are certain cells that you don't want to include in the summary, you can add the `#| exclude` parameter to exclude them.

In [ ]:
#| exclude
def multiply_numbers(a, b):
    return a * b


In [ ]:
%%prompt
Do we have any multiplication functions in this notebook?

No, there are no multiplication functions specifically mentioned or defined in the current notebook context. If you would like to add one, I can help you create a multiplication function. Let me know how you'd like to proceed!

You can also check the usage of the last prompt.

In [ ]:
last_prompt_result.usage()

Usage(requests=1, request_tokens=4086, response_tokens=45, total_tokens=4131, details={'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0, 'cached_tokens': 3840})
